# 第4回 FUN AI ハンズオンkaggle部門
公立はこだて未来大学　FUN AIサークルで使う資料です。B1加藤まる

This is a material used at the Future Hakodate Future University FUN AI Circle Hands-on.
## ～結果を予測する～
前回、訓練用データをいじってヒートマップを表示させやした。今回はscikitlearnにあるモデルを訓練して、結果を予測します。以下手順。
1. テストデータも訓練データと同じ処理をする
2. "train_test_split"を使って訓練データを更に訓練データとテストデータにわける
3. モデルを訓練させ、スコアを出す
4. スコアの高いモデルでテストデータから結果を予測する。
5. 提出！

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn import preprocessing

In [4]:
import pandas as pd
import os 
dataset_p = "datasets"

gender_submission = pd.read_csv(os.path.join(dataset_p, "gender_submission.csv"))
test = pd.read_csv(os.path.join(dataset_p, "test.csv"))
train = pd.read_csv(os.path.join(dataset_p, "train.csv"))



In [6]:
train['Cabin'].fillna(train['Cabin'].mode()[0],inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0],inplace=True)
train['Age'].fillna(train['Age'].mode()[0],inplace=True)
train.drop(columns=['Name','Ticket'],inplace=True)
train['Fare'].fillna(train['Fare'].mode()[0],inplace=True)

for column in ['Sex','Cabin','Embarked']:
    le = preprocessing.LabelEncoder()
    le.fit(train[column])
    train[column] = le.transform(train[column])
    
full_data = [train]
for train in full_data:
    train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

train.drop(columns=['SibSp','Parch'],inplace=True)
train.drop(columns='PassengerId',inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
Fare          891 non-null float64
Cabin         891 non-null int64
Embarked      891 non-null int64
FamilySize    891 non-null int64
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [7]:
test['Cabin'].fillna(test['Cabin'].mode()[0],inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0],inplace=True)
test['Age'].fillna(test['Age'].mode()[0],inplace=True)
test.drop(columns=['Name','Ticket'],inplace=True)
test['Fare'].fillna(test['Fare'].mode()[0],inplace=True)

for column in ['Sex','Cabin','Embarked']:
    le = preprocessing.LabelEncoder()
    le.fit(test[column])
    test[column] = le.transform(test[column])

full_data = [test]
for test in full_data:
    test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

test.drop(columns=['SibSp','Parch'],inplace=True)
test.drop(columns='PassengerId',inplace=True)
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
Pclass        418 non-null int64
Sex           418 non-null int64
Age           418 non-null float64
Fare          418 non-null float64
Cabin         418 non-null int64
Embarked      418 non-null int64
FamilySize    418 non-null int64
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


In [8]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,FamilySize
0,0,3,1,22.0,7.2500,47,2,2
1,1,1,0,38.0,71.2833,81,0,2
2,1,3,0,26.0,7.9250,47,2,1
3,1,1,0,35.0,53.1000,55,2,2
4,0,3,1,35.0,8.0500,47,2,1


In [9]:
from sklearn.model_selection import train_test_split
Y_train = train['Survived']
X_train = train.drop(['Survived'], axis=1)

X_train , X_test , Y_train , Y_test = train_test_split(X_train , Y_train ,test_size = 0.1,random_state = 0)

In [10]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_test, Y_test) * 100, 2)
acc_log

/Users/hamaguchikazuki/.local/share/virtualenvs/titanic-jNgzaQGt/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


83.33

ロジスティック回帰を使用して、特徴量を作成して目標を達成するための仮定と決定を検証できます。 これは、決定関数の特徴の係数を計算することで実行できます。FamilySize作ったけどあんま良い特徴量じゃなかったな～などと。

In [11]:
coeff_df = pd.DataFrame(train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
4,Cabin,0.008646
3,Fare,0.003194
2,Age,-0.038043
5,Embarked,-0.179363
6,FamilySize,-0.207457
0,Pclass,-0.947305
1,Sex,-2.556294


分類および回帰分析に使用されるデータを分析する関連学習アルゴリズムを備えた教師あり学習モデルであるサポートベクターマシン
2つのカテゴリのいずれか一方に属するとマークされたトレーニングサンプルのセットが与えられると、新しいテストサンプルを２つのカテゴリに割り当てるモデルを構築し、非確率的バイナリ線形分類器にします。 

In [12]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [13]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
acc_svc = round(svc.score(X_test, Y_test) * 100, 2)
acc_svc

70.0

パターン認識では、k-Nearest Neighborsアルゴリズム（またはk-NN）は、分類と回帰に使用されるノンパラメトリック手法です。 サンプルは、その近傍の多数決によって分類されます。サンプルは、k個の最近傍（kは正の整数、通常は小さい）の中で最も一般的なクラスに割り当てられます。 k = 1の場合、オブジェクトはその単一の最近傍のクラスに単純に割り当てられます。

In [14]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
acc_knn

74.44

機械学習では、ナイーブベイズ分類器は、ベイズの定理を特徴間の強い（単純な）独立仮定で適用することに基づく単純な確率的分類器のファミリーです。 Naive Bayes分類器は非常にスケーラブルであり、学習問題の変数（特徴）の数に線形のパラメーターが必要です。

In [15]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
acc_gaussian = round(gaussian.score(X_test, Y_test) * 100, 2)
acc_gaussian

82.22

パーセプトロンは、バイナリ分類器（数値のベクトルで表される入力が特定のクラスに属するかどうかを決定できる関数）の教師あり学習のためのアルゴリズムです。 これは、線形分類器の一種です。つまり、重みのセットを特徴ベクトルと組み合わせた線形予測関数に基づいて予測を行う分類アルゴリズムです。 このアルゴリズムでは、トレーニングセットの要素を1つずつ処理するという点で、オンライン学習が可能です。

In [16]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
acc_perceptron = round(perceptron.score(X_test, Y_test) * 100, 2)
acc_perceptron

43.33

In [17]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
acc_linear_svc = round(linear_svc.score(X_test, Y_test) * 100, 2)
acc_linear_svc

/Users/hamaguchikazuki/.local/share/virtualenvs/titanic-jNgzaQGt/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


56.67

In [18]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
acc_sgd = round(sgd.score(X_test, Y_test) * 100, 2)
acc_sgd

71.11

このモデルでは、意思決定ツリーを予測モデルとして使用し、機能（ツリーブランチ）をターゲット値（ツリーリーフ）に関する結論にマップします。 ターゲット変数が有限の値のセットを取ることができるツリーモデルは、分類ツリーと呼ばれます。 これらのツリー構造では、リーフはクラスラベルを表し、ブランチはそれらのクラスラベルにつながる機能の組み合わせを表します。 ターゲット変数が連続値（通常は実数）を取ることができる決定木は、回帰木と呼ばれます。

In [19]:

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
acc_decision_tree = round(decision_tree.score(X_test, Y_test) * 100, 2)
acc_decision_tree

81.11

次のモデルのランダムフォレストは、最も人気のあるモデルの1つです。 ランダムフォレストまたはランダム決定フォレストは、分類、回帰、およびその他のタスクのためのアンサンブル学習方法であり、トレーニング時に多数の決定ツリー（n_estimators = 100）を構築し、クラスのモードであるクラスを出力することにより動作します（分類） または、個々のツリーの平均予測（回帰）。

In [20]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_test, Y_test) * 100, 2)
acc_random_forest

80.0

In [21]:
Y_pred = random_forest.predict(test)

In [22]:
Y_pred

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,

In [23]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission.csv', index=False)

NameError: name 'test_df' is not defined